In [6]:
import os
import boto3
import sys
from prophet import Prophet
import matplotlib.pyplot as plt
import datetime
import pandas as pd



In [14]:
results = []

def history(client, instance, region_):
    INSTANCE = instance
    STARTTIME = (datetime.datetime.now() - datetime.timedelta(days=60)).isoformat()
    
    prices = client.describe_spot_price_history(
        InstanceTypes=[INSTANCE],
        ProductDescriptions=['Linux/UNIX'],
        AvailabilityZone="{region_}a".format(region_= region_),
        StartTime=STARTTIME,
        MaxResults=200
    )
    # print("Prices for region %s: %s" % (region, prices["SpotPriceHistory"]))
    for price in prices["SpotPriceHistory"]:
        results.append({
                        'Timestamp': price["Timestamp"].strftime('%m-%d-%Y'), 
                        'Price': price["SpotPrice"]
                        })

        
    df = pd.DataFrame.from_dict(results)
    df = df.rename(columns={'Timestamp': 'ds', 'Price': 'y'})
    df = df.sort_values('y', ascending=False).drop_duplicates('ds').sort_values('ds').reset_index(drop=True)
    
    return df

In [15]:
def interpolate(df):
    # Split the data into train and test sets
    # Train the model on the train set
    model = Prophet(
        daily_seasonality=True,
        yearly_seasonality=False,
    )
    model.fit(df)

    # Predict for the test set
    future = model.make_future_dataframe(periods=1)
    forecast = model.predict(future)
    predicted_prices = forecast['yhat'].iloc[-1]

    return predicted_prices

In [16]:
def main(instance):
    region = "us-east-1"
    client = boto3.client('ec2', region_name=region)
    df = history(client, instance, region)
    print(df)
    for i in range(len(df) - 21):
        print(f'Itertaion: {i}')
        new_df = df.iloc[i:i+21]
        print(f' new df start: {new_df.iloc[0]}')
        print(f' new df end: {new_df.iloc[-1]}')
        price = interpolate(new_df)
        df.loc[i+21, 'yhat'] = price
    print(df)
    return df

In [17]:
# spots = {"t2.small": {"cpu": 1, "memory": 2, "date": "2023-03-22", "cost": 0.008}, "t3.medium": {"cpu": 2, "memory": 4, "date": "2023-03-19", "cost": 0.019}, "m6g.medium": {"cpu": 1, "memory": 4, "date": 22, "cost": 0.01}, "c6a.large": {"cpu": 2, "memory": 4, "date": 22, "cost": 0.01}, "t4g.large": {"cpu": 2, "memory": 8, "date": 22, "cost": 0.01}, "c6g.xlarge": {"cpu": 4, "memory": 8, "date": 22, "cost": 0.01}}
# instances = list(spots.keys())
# print(instances)
# for i in instances:
#   print(f'Instance: {i}')
#   main(i)

updated_df = main('t2.small')


INFO:prophet:n_changepoints greater than number of observations. Using 15.


            ds         y
0   03-18-2023  0.007800
1   03-19-2023  0.007900
2   03-20-2023  0.008000
3   03-21-2023  0.008200
4   03-22-2023  0.008400
..         ...       ...
56  05-13-2023  0.017500
57  05-14-2023  0.017400
58  05-15-2023  0.017300
59  05-16-2023  0.017600
60  05-17-2023  0.017300

[61 rows x 2 columns]
Itertaion: 0
 new df start: ds    03-18-2023
y       0.007800
Name: 0, dtype: object
 new df end: ds    04-07-2023
y       0.013100
Name: 20, dtype: object


INFO:prophet:n_changepoints greater than number of observations. Using 15.


Itertaion: 1
 new df start: ds      03-19-2023
y         0.007900
yhat           NaN
Name: 1, dtype: object
 new df end: ds      04-08-2023
y         0.013700
yhat      0.013458
Name: 21, dtype: object


INFO:prophet:n_changepoints greater than number of observations. Using 15.


...

Itertaion: 37
 new df start: ds      04-24-2023
y         0.017400
yhat      0.018039
Name: 37, dtype: object
 new df end: ds      05-14-2023
y         0.017400
yhat        0.0176
Name: 57, dtype: object


INFO:prophet:n_changepoints greater than number of observations. Using 15.


Itertaion: 38
 new df start: ds      04-25-2023
y         0.017400
yhat      0.018146
Name: 38, dtype: object
 new df end: ds      05-15-2023
y         0.017300
yhat      0.017554
Name: 58, dtype: object


INFO:prophet:n_changepoints greater than number of observations. Using 15.


Itertaion: 39
 new df start: ds      04-26-2023
y         0.017300
yhat      0.017529
Name: 39, dtype: object
 new df end: ds      05-16-2023
y         0.017600
yhat      0.017232
Name: 59, dtype: object
            ds         y      yhat
0   03-18-2023  0.007800       NaN
1   03-19-2023  0.007900       NaN
2   03-20-2023  0.008000       NaN
3   03-21-2023  0.008200       NaN
4   03-22-2023  0.008400       NaN
..         ...       ...       ...
56  05-13-2023  0.017500  0.017574
57  05-14-2023  0.017400  0.017600
58  05-15-2023  0.017300  0.017554
59  05-16-2023  0.017600  0.017232
60  05-17-2023  0.017300  0.017517

[61 rows x 3 columns]


In [18]:
updated_df

,ds,y,yhat
0,03-18-2023,0.007800,NaN
1,03-19-2023,0.007900,NaN
2,03-20-2023,0.008000,NaN
3,03-21-2023,0.008200,NaN
4,03-22-2023,0.008400,NaN
...,...,...,...
56,05-13-2023,0.017500,0.017574
57,05-14-2023,0.017400,0.017600
58,05-15-2023,0.017300,0.017554
59,05-16-2023,0.017600,0.017232


In [19]:
def plot2(df):
    # Create a figure and axis
    fig, ax = plt.subplots()
    print(df)

    # Plot the 'y' and 'y_hat' columns against the 'date' column
    ax.plot(df['ds'], df['y'], label='Actual')
    ax.plot(df['ds'], df['yhat'], label='Predicted')

    # Set the x-axis label
    ax.set_xlabel('Date')

    # Set the y-axis label
    ax.set_ylabel('Spot Price')

    # Set the title of the plot
    ax.set_title('Plot of actual and predicted')

    # Display a legend
    ax.legend()

    # Rotate x-axis labels for better readability (optional)
    plt.xticks(rotation=45)

    # Display the plot
    plt.show()

In [20]:
plot2(updated_df)

INFO:matplotlib.category:Using categorical units to plot a list of strings that are all parsable as floats or dates. If these strings should be plotted as numbers, cast to the appropriate data type before plotting.
INFO:matplotlib.category:Using categorical units to plot a list of strings that are all parsable as floats or dates. If these strings should be plotted as numbers, cast to the appropriate data type before plotting.
INFO:matplotlib.category:Using categorical units to plot a list of strings that are all parsable as floats or dates. If these strings should be plotted as numbers, cast to the appropriate data type before plotting.
INFO:matplotlib.category:Using categorical units to plot a list of strings that are all parsable as floats or dates. If these strings should be plotted as numbers, cast to the appropriate data type before plotting.
INFO:matplotlib.category:Using categorical units to plot a list of strings that are all parsable as floats or dates. If these strings should

            ds         y      yhat
0   03-18-2023  0.007800       NaN
1   03-19-2023  0.007900       NaN
2   03-20-2023  0.008000       NaN
3   03-21-2023  0.008200       NaN
4   03-22-2023  0.008400       NaN
..         ...       ...       ...
56  05-13-2023  0.017500  0.017574
57  05-14-2023  0.017400  0.017600
58  05-15-2023  0.017300  0.017554
59  05-16-2023  0.017600  0.017232
60  05-17-2023  0.017300  0.017517

[61 rows x 3 columns]


In [21]:
updated_df.to_csv('t2_small2.csv', index=False)